In [13]:
### ENV: pipy135
import pandas as pd
import fastai as fastai
import numpy as np
from fastai import tabular
from fastai.imports import *
from fastai import *
from fastai.tabular import *
from fastai.tabular.transform import Categorify, FillMissing, Normalize, add_datepart
from fastai.tabular.data import TabularDataBunch, TabularList
from sklearn import metrics
import sklearn as sk
from fastai.metrics import accuracy, rmse
from fastai.tabular.learner import tabular_learner

In [14]:
### INPUTS TO FASTAI FOR DEALING WITH MISSING VALUES, CATEGORICAL AND NUMERIC FEATURES
procs = [FillMissing,Categorify,Normalize]
procs1 = [FillMissing,Categorify]

In [15]:
### IMPORT THE FILE WITH DATASET
inpDf = pd.read_csv('/data2/home/prasannaiyer/Projects/TT_Fastai/Dataset/OTD_2019_PU.csv')
inpDf = inpDf[(inpDf['Days Late']<20)&((inpDf['Location Type']=='Plant')\
    |(inpDf['Location Type']=='Port')|(inpDf['Location Type']=='Keen'))]

In [16]:
### RENAME COLUMNS AS WELL AS SET DATATYPE OF DATE FEATURES  ###
inpDf['Delivery_Date'] = pd.to_datetime(inpDf['Shipment End Date'])
inpDf['Pickup_Date'] = pd.to_datetime(inpDf['First shipped date (first YT26 action) Date'])
inpDf.rename(columns = {'Shipment Secure Resources Upd Dt':'Tender_Date'},inplace = True)
inpDf['Tender_Date'] = pd.to_datetime(inpDf['Tender_Date'])

In [17]:
### CREATE NEW FEATURES
### OutputTransitTime IS THE DEPENDENT VARIABLE
### OD IS A FEATURE FOR OVER-DIMENTIONSAL SHIPMENTS
inpDf['OutputTransitTime'] = (inpDf['Delivery_Date']-inpDf['Pickup_Date']).dt.days
inpDf['OD'] = [1 if OD > 0 else 0 for OD in inpDf['Shipment OD Day']]

In [18]:
### REMOVE OUTLIERS BASED ON CRITERIA FROM EDA ###
inpDf = inpDf[(inpDf['OutputTransitTime'] <16) & (inpDf['OutputTransitTime'] > 0)]
inpDf = inpDf[(inpDf['Shipment Loaded Distance']>200) & (inpDf['Shipment Loaded Distance']<2501)]
### DROP NULL VALUES ###
inpDf.dropna(subset=['OutputTransitTime'],inplace=True)
#inpDf['OutputTransitTime'] = inpDf['OutputTransitTime'].astype('int32')

In [19]:
### DROP COLUMNS NOT NEEDED FOR PREDICTION ###
inpDf.drop(['Shipment GID', 'Delivery Status','Shipment Source Location GID',\
    'Shipment Source Province Code','Shipment Destination Location GID',\
        'Shipment Destination Location Name', 'Shipment Destination City',\
            'Shipment Enroute Status Upd Dt','Vehicle Outbound Delivery tendered and accepted Date',\
                'First shipped date (first YT26 action) Date', 'Shipment End Date',\
                    'Expected Days to Deliver', 'Actual Days', 'Days Late','Year',\
                        'Month','Shipment OD Day','Pickup_Date','Delivery_Date'],\
                            inplace = True,axis=1)

In [20]:
### CREATION OF NEW FEATURES FROM EXISTING DATE FEATURES ###
inpDf = add_datepart(inpDf,'Tender_Date',drop=False)

In [21]:
### SORT THE DATAFRAME ###
### Next step is to split the data into training and testing dataset. 
### Since this data involves dates, dataset is sorted by the specific date feature. 
### This ensures that older data is used for training and later data is used in testing. 
### This is important as the model will be used to predict future values of the dependent variable
inpDf = inpDf.sort_values(by=['Tender_Date'])
inpDf = inpDf.drop(['Tender_Date'],axis=1)

In [22]:
### CATEGORICAL COLUMNS ###
cat_columns = []
for c in inpDf.columns:
  if inpDf[c].dtype in ['object']:
    cat_columns.append(c) 

In [23]:
### CONTINUOUS OR NUMERICAL COLUMNS ###
cont_columns = []
for c in inpDf.columns:
  if inpDf[c].dtype not in ['object']:
    cont_columns.append(c) 

In [24]:
### CREATE EMBEDDINGS ###
emb_sizes={}
for c in cat_columns:
    if inpDf[c].dtypes == 'O':
        cat_count = int(inpDf[c].nunique()/2)
        if cat_count>1:
            emb_sizes.update({c:cat_count})

In [25]:
### BOOLEAN COLUMNS ###
for bcol in cont_columns:
    #print(bcol,'--',inpDf[bcol].dtypes)
    if inpDf[bcol].dtypes == 'bool':
        inpDf[bcol] = inpDf[bcol].astype('int32')

In [26]:
### TRAIN/TEST DATA SPLIT ###
data_count = inpDf.shape[0]
data_train_count = int(data_count*0.8)
data_val_count = data_count - data_train_count
train_data = inpDf[:data_train_count]
test_data = inpDf[data_train_count:]
val_idx = range(data_train_count,data_count-1)
val_idx1 = range(data_train_count,data_count)

In [27]:
### Training & prediction
dep_var = 'OutputTransitTime'
cont_columns.remove(dep_var)
path = ''
tab_databunch1 = TabularDataBunch.from_df(path,inpDf,dep_var,valid_idx=val_idx1,\
    cat_names=cat_columns,procs = procs1,cont_names=cont_columns) 

learner1 = tabular_learner(tab_databunch1,layers=[1000,500],\
     emb_szs=emb_sizes,metrics=rmse)

### Train 20 epochs
learner1.fit_one_cycle(10, 1e-2)

### Prediction details
pred1,y1,loss1 = learner1.get_preds(with_loss=True)
x = rmse(pred1,y1)

epoch,train_loss,valid_loss,root_mean_squared_error,time
0,13.380310,15.756706,3.281795,00:06
1,7.188285,6.622568,2.459757,00:06
2,6.880220,5.703304,2.265379,00:06
3,6.574411,5.318990,2.218133,00:06
4,6.291667,5.077841,2.180535,00:06
5,5.952751,4.785054,2.125625,00:06
6,5.374725,5.842860,2.318283,00:06
7,4.833013,5.010554,2.153146,00:06
8,4.451330,5.014889,2.148119,00:06
9,4.264637,4.918071,2.141319,00:06
